In [1]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import nltk
import pandas as pd
from nltk.stem import WordNetLemmatizer
from nltk.parse.corenlp import CoreNLPDependencyParser
from nltk.parse.corenlp import CoreNLPParser
from nltk.corpus import wordnet as wn
from nltk.probability import FreqDist
from collections import Counter
import math
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

# XGBOOST
import xgboost as xgb
# from xgboost import XGBClassifier

# SPACY IMPORT
import spacy
nlp = spacy.load("en_core_web_md")

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [2]:
from xgboost import XGBClassifier

In [3]:
# same readData from STS.py
def readData(fileName):

    first_sentence = []
    second_sentence = []
    score = []
    file = open(fileName, encoding="utf8")
    text = file.readline()
    text = file.read()
    # loop to extract a set of two sentences
    for sentence in text.split('\n'):
        # creating two separate lists of the sentences
        # '.rstrip('.') only removes the last period in the sentence
        first_sentence.insert(len(first_sentence),
                              (sentence.split('\t')[1].lower()).rstrip('.'))
        second_sentence.insert(len(first_sentence),
                               (sentence.split('\t')[2].lower()).rstrip('.'))
        # inserting the score as a separate lists
        score.insert(len(first_sentence), (sentence.split('\t')[3]))

    # print(first_sentence)
    return first_sentence, second_sentence, score


def preprocess(fileName):

    first_sentence, second_sentence, score = readData(fileName)
    first_sentence_tokens = []
    second_sentence_tokens = []

    # tokenizing and tagging
    first_sentence_tags = []
    second_sentence_tags = []

    for sentence in first_sentence:
        tokens = nltk.word_tokenize(sentence)
        first_sentence_tokens.insert(len(first_sentence_tokens), tokens)
        first_sentence_tags.insert(
            len(first_sentence_tags), nltk.pos_tag(tokens))
        # print(first_sentence_tokens)

    for sentence in second_sentence:
        tokens = nltk.word_tokenize(sentence)
        second_sentence_tokens.insert(len(second_sentence_tokens), tokens)
        second_sentence_tags.insert(
            len(second_sentence_tags), nltk.pos_tag(tokens))

        # print(second_sentence_tokens)

    # lemmatizing
    first_sentence_lemmas = []
    second_sentence_lemmas = []
    lemmatizer = WordNetLemmatizer()
    for sentence in first_sentence_tokens:
        sentence_components = []
        for token in sentence:
            lemmas = lemmatizer.lemmatize(token)
            sentence_components.insert(len(sentence_components), lemmas)
        first_sentence_lemmas.insert(
            len(first_sentence_lemmas), sentence_components)

    for sentence in second_sentence_tokens:
        sentence_components = []
        for token in sentence:
            lemmas = lemmatizer.lemmatize(token)
            sentence_components.insert(len(sentence_components), lemmas)
        second_sentence_lemmas.insert(
            len(second_sentence_lemmas), sentence_components)

    return first_sentence, second_sentence, score, first_sentence_tokens, second_sentence_tokens, first_sentence_lemmas, second_sentence_lemmas


In [4]:
s1_arr_train, s2_arr_train, scores_train, s1_tokens_train, s2_tokens_train, s1_lemmas_train,s2_lemmas_train = preprocess("./data/train-set.txt")

## Data Inspection

In [5]:
score_list = [0,0,0,0,0,0]
for s in scores_train:
    score_list[int(s)] += 1
for i in range(0, len(score_list)):
#     print(i, ': ', score_list[i], round(score_list[i]/len(score_list), 1), '%')
    print("% 1d: % 4d % 6.2f per" %(i, score_list[i], 100*score_list[i]/len(scores_train))) 

 0:    8   0.54 per
 1:   37   2.49 per
 2:   95   6.40 per
 3:  310  20.89 per
 4:  616  41.51 per
 5:  418  28.17 per


## Feature Engineering

This section includes all the code/functions to create features.

### Cosine Similarity (TF-IDF)

In [6]:
def calc_cosine_similarity(sentence1, sentence2):

    # remove the stopwords, transform into TF-IDF matrix, then
    tfidf_matrix = TfidfVectorizer(
        stop_words="english").fit_transform([sentence1, sentence2])
    cos_sim_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)
#     print(tfidf_matrix.toarray())

    cos_sim = cos_sim_matrix[0][1]

    return cos_sim

### Smooth Inverse Frequency (SIF)

In [7]:
def frequency_distribution(s1_tokens_array, s2_tokens_array):
    freq_dist = FreqDist()
    for i in range(len(s1_tokens_array)):
        for token in (s1_tokens_array[i] + s2_tokens_array[i]):
            freq_dist[token.lower()] += 1
    return freq_dist

In [8]:
freq_dist = frequency_distribution(s1_tokens_train, s2_tokens_train)
print(freq_dist.most_common(40))

[('the', 5169), (',', 3690), ('of', 2497), ('to', 2133), ('and', 1716), ('a', 1615), ('in', 1573), ('is', 891), ('that', 831), ('on', 820), ('for', 756), ('it', 587), ('this', 579), ('we', 531), ('with', 464), ('be', 459), ('by', 443), ('i', 425), ('which', 403), ('have', 384), ('not', 366), ('at', 343), ('as', 334), ('are', 333), ('has', 319), ('said', 316), ('was', 304), ('european', 287), ("'s", 280), ('from', 261), ('``', 252), ("''", 242), ('will', 233), ('.', 229), ('also', 223), ('its', 194), ('but', 193), ('would', 191), ('all', 188), ('percent', 187)]


In [9]:
def calc_sif_similarity(s1, s2, a = .001):
    vectorizer = CountVectorizer(stop_words="english")
    X = vectorizer.fit_transform([s1, s2])
    X_arr = X.toarray()
    sif_matrix = []
    for i in range(0, len(X_arr)):
        sif_arr = []
        for j in range(0, len(X_arr[i])):
            word = vectorizer.get_feature_names()[j]
            w = a / (a + freq_dist[word])
            v = X_arr[i][j]
            sif_arr.append(v*w)
        sif_matrix.append(sif_arr)
    sif_cos_sim_matrix = cosine_similarity(sif_matrix, sif_matrix)
    sif_cos_sim = sif_cos_sim_matrix[0][1]
    return sif_cos_sim

In [10]:
calc_sif_similarity('I like some apples', 'I like the pears')

1.4515545128534995e-10

### Simple Overlap

Unique words that are in both sentences divided by the total number of words in both sentences. Does not include stop words.

In [11]:
stop_words = set(stopwords.words('english'))
tokenized_sentence_list = s1_tokens_train+s2_tokens_train
words_filtered = []

# print(words)

# looking through I've noticed there are a number of stop-words that can be added to the set
stop_words.add(',')
stop_words.add('``')
stop_words.add("n't")

for tsl in tokenized_sentence_list:
    for w in tsl:
        if w not in stop_words and w not in words_filtered:
            words_filtered.append(w)

In [12]:
def remove_stopwords(token_list):
    blank_list = []
    for w in token_list:
        if w not in stop_words:
            blank_list.append(w)
    return blank_list

In [13]:
def remove_duplicate_tokens(token_list):
    blank_list = []
    for w in token_list:
        if w not in blank_list:
            blank_list.append(w)
    return blank_list

In [14]:
def calc_basic_overlap(s1_tokens, s2_tokens):
    s1_tokens = remove_stopwords(s1_tokens)
    s1_tokens = remove_duplicate_tokens(s1_tokens)

    s2_tokens = remove_stopwords(s2_tokens)
    s2_tokens = remove_duplicate_tokens(s2_tokens)
    
    overlap = 0
    encountered_indexes = []
    for word in (s1_tokens+s2_tokens):
        try:
            word_index = words_filtered.index(word)
            if word_index in encountered_indexes: # we know we have found an overlap
                overlap += 1
            encountered_indexes.append(word_index)
        except ValueError:
            # print(word + ' not found in lexicon. Skipping...')
            continue

    avg_sentence_len = len(s1_tokens+s2_tokens) / 2
    
    overlap_normlalized = overlap / avg_sentence_len
    return overlap, overlap_normlalized

### Synset Overlap

In [15]:
def calc_synset_overlap(s1_tokens, s2_tokens):
    s1_tokens = remove_stopwords(s1_tokens)
    s1_tokens = remove_duplicate_tokens(s1_tokens)

    s2_tokens = remove_stopwords(s2_tokens)
    s2_tokens = remove_duplicate_tokens(s2_tokens)
    
    print(s2_tokens)
    print(s1_tokens)

    s1_spread = []
    s2_spread = []
    
    for word in s1_tokens:
        for synset in wn.synsets(word):
            for i in range(0, len(synset.lemmas())):
                syn_word = synset.lemmas()[i].name()
                if syn_word not in s1_spread:
                    s1_spread.append(syn_word)

    for word in s2_tokens:
        for synset in wn.synsets(word):
            for i in range(0, len(synset.lemmas())):
                syn_word = synset.lemmas()[i].name()
                if syn_word not in s2_spread:
                    s2_spread.append(syn_word)         
    
    return calc_basic_overlap(s1_spread, s2_spread)
    
calc_synset_overlap(s1_tokens_train[0], s2_tokens_train[0])

['sources', 'close', 'sale', 'said', 'vivendi', 'keeping', 'door', 'open', 'bids', 'next', 'day', 'two']
['sources', 'close', 'sale', 'said', 'vivendi', 'keeping', 'door', 'open', 'bids', 'hoped', 'see', 'bidders', 'interested', 'individual', 'assets', 'team']


(46, 0.23173803526448364)

### Spacy Similarity

In [16]:
def calc_spacy_sim(s1, s2):
    s2 = nlp(s2)
    s1 = nlp(s1)
    return s1.similarity(s2)


## Pipeline

In this section we run the data through the pipeline to get it into the form necessary to create our models.

In [17]:
def pipeline(s1_array, s2_array, s1_tokens, s2_tokens, s1_lemmas, s2_lemmas):
    # TODO add a check to ensure the lengths of these arrays are the same
    # or add the basic processing to pipeline
    data = []
    for i in range(0, len(s1_array)):
        cos_sim = calc_cosine_similarity(s1_array[i], s2_array[i])
        sif_sim = calc_sif_similarity(s1_array[i], s2_array[i])
        w_overlap, w_norm_overlap = calc_basic_overlap(s1_tokens[i], s2_tokens[i])
        l_overlap, l_norm_overlap = calc_basic_overlap(s1_lemmas[i], s2_lemmas[i])
        spacy_sim = calc_spacy_sim(s1_array[i], s2_array[i])
        syn_overlap, normalized_syn_overlap = calc_synset_overlap(s1_tokens[i], s2_tokens[i])
        data.append([i, w_norm_overlap, l_norm_overlap, sif_sim, cos_sim, spacy_sim, syn_overlap, normalized_syn_overlap])
    return data

In [18]:
data = pipeline(s1_arr_train, s2_arr_train, s1_tokens_train, s2_tokens_train, s1_lemmas_train, s2_lemmas_train)
print(data[0:5])

['sources', 'close', 'sale', 'said', 'vivendi', 'keeping', 'door', 'open', 'bids', 'next', 'day', 'two']
['sources', 'close', 'sale', 'said', 'vivendi', 'keeping', 'door', 'open', 'bids', 'hoped', 'see', 'bidders', 'interested', 'individual', 'assets', 'team']
['micron', "'s", 'numbers', 'also', 'marked', 'first', 'quarterly', 'profit', 'three', 'years', 'dram', 'manufacturer']
['micron', 'declared', 'first', 'quarterly', 'profit', 'three', 'years']
['perry', 'said', 'backs', 'senate', "'s", 'efforts', 'including', 'fines', 'force', 'democrats', 'return']
['fines', 'part', 'failed', 'republican', 'efforts', 'force', 'entice', 'democrats', 'return']
['american', 'anglican', 'council', 'represents', 'episcopalian', 'conservatives', 'said', 'seek', 'authorization', 'create', 'separate', 'province', 'north', 'america', 'last', 'week', "'s", 'actions']
['american', 'anglican', 'council', 'represents', 'episcopalian', 'conservatives', 'said', 'seek', 'authorization', 'create', 'separate', 'g

['neither', 'peace', 'rules', 'could', 'keep', 'funny', 'cide', 'drawing', 'away']
['peace', 'rules', 'defeated', 'funny', 'cide', 'louisiana', 'derby']
['waksal', 'pleaded', 'guilty', 'insider', 'trading', 'charges', 'last', 'year', 'scheduled', 'sentenced', 'june', '10']
['waksal', 'pleaded', 'guilty', 'securities', 'fraud', 'sentenced', 'next', 'week']
['zambrano', 'pitched', 'seven', 'innings', 'allowed', 'two', 'runs', 'five', 'hits', 'four', 'walks']
['allowed', 'two', 'runs', 'seven', 'innings', 'struck', 'six']
["'m", 'absolutely', 'confident', "'re", 'going', 'bill', "''", 'frist', 'r-tenn.', 'said', 'thursday']
['still', 'said', "'m", 'absolutely', 'confident', "'re", 'going', 'bill', '.', "''"]
['korean', 'air', 'deal', 'expected', 'finalized', 'next', 'several', 'weeks', "''", 'boeing', 'spokesman', 'bob', 'saling', 'said']
['boeing', 'said', 'final', 'agreement', 'expected', 'signed', 'next', 'weeks']
['shares', 'eds', 'closed', 'thursday', '$', '18.51', '6', 'cents', 'new

['wire', 'transfers', 'four', '40', 'accounts', 'open', 'beacon', 'hill', 'totaled', '$', '3.2', 'billion', '2001', '2002', 'morgenthau', 'said']
['2001', '2002', 'morgenthau', 'said', 'wire', 'transfers', 'four', 'beacon', 'hill', "'s", '40', 'accounts', 'totaled', '$', '3.2', 'billion']
['ran', 'last', 'year', 'democratic', 'nomination', 'texas', 'governor', 'lost', 'primary', 'multimillionaire', 'tony', 'sanchez']
['last', 'year', 'made', 'unsuccessful', 'bid', 'democratic', 'nomination', 'governor']
['wal-mart', 'told', 'top', '100', 'suppliers', "'ll", 'need', 'radio-frequency', 'id', 'systems', 'place', 'tracking', 'pallets', 'goods', 'supply', 'chain', 'jan.', '25', '2005']
['nation', "'s", 'largest', 'retailer', 'told', '100', 'top', 'suppliers', 'start', 'using', 'electronic', 'tags', 'pallets', 'goods', 'jan.', '25', '2005']
['harvey', 'taken', 'st.', 'luke', "'s", 'hospital', 'precautionary', 'neck', 'x-rays', 'came', 'back', 'negative']
['taken', 'hospital', 'precautionary'

['palm', 'said', 'wednesday', 'plans', 'buy', 'handspring', 'company', 'created', 'renegade', 'co-founder', 'jeff', 'hawkins']
['palm', 'wednesday', 'announced', 'plans', 'acquire', 'handspring', 'company', 'started', 'jeff', 'hawkins', 'regarded', 'many', 'father', 'handheld']
["'m", 'going', 'sponsoring', "'s", 'proposal', 'responding', 'negative', 'way', "''", 'said']
["'m", 'going', 'sponsoring', 'proposal', 'negative', "''"]
['government', 'recently', 'shelved', 'peace', 'talks', 'brokered', 'neighbouring', 'malaysia', 'spate', 'attacks', 'mindanao', 'including', 'three', 'deadly', 'bombings', 'blamed', 'milf']
['government', 'recently', 'shelved', 'peace', 'talks', 'milf', 'brokered', 'malaysia', 'string', 'attacks', 'including', 'three', 'bombings', 'mindanao']
['year-ago', 'quarter', 'company', 'earned', '$', '54.3', 'million', '22', 'cents', 'share']
['company', 'posted', 'profit', '$', '54.3', 'million', '22', 'cents', 'per', 'share', 'year-ago', 'period']
['along', 'boston',

['dynes', 'uc', 'san', 'diego', 'since', '1991', 'spending', '22', 'years', '&', 'bell', 'labs', 'worked', 'superconductors', 'materials']
['dynes', 'came', 'uc', 'san', 'diego', '1991', '22', 'years', 'physicist', '&', 'bell', 'labs']
['shares', 'ba', 'three', 'percent', '165-1/4', 'pence', '0933', 'gmt', 'low', '164', 'stronger', 'market']
['shares', 'ba', '1.5', 'percent', '168', 'pence', '1420', 'gmt', 'low', '164p', 'slightly', 'stronger', 'overall', 'london', 'market']
['house', 'senate', 'bills', 'would', 'end', 'ban', 'research', 'development', 'low-yield', 'nuclear', 'weapons']
['senate', 'agreed', 'tuesday', 'lift', '10-year-old', 'ban', 'research', 'development', 'low-yield', 'nuclear', 'weapons']
['meningitis', 'infection', 'fluid', 'person', "'s", 'spinal', 'cord', 'around', 'brain']
['meningitis', 'infection', 'spinal', 'cord', 'fluid', 'tissue', 'around', 'brain']
['chiron', 'aventis', 'pasteur', 'together', 'made', '80', 'million', 'doses', 'ordinarily', 'enough', 'u.s.

...

['secondly', 'must', 'say', "'no", "'", 'concept', 'bang', 'would', 'like', 'see', 'time', 'admit', 'large', 'group', 'countries', 'contradiction', 'principle', 'admission', 'country', 'according', 'merits']
['secondly', 'say', 'big', 'bang', 'idea', 'incorporating', 'candidate', 'countries', 'simultaneously', 'large', 'group', 'since', 'would', 'militate', 'principle', 'country', 'admitted', 'merits']
['course', 'important', 'point', 'fact', 'started', 'heading', 'four', 'approach', 'know', 'yet', 'well', 'managed', 'solve', 'problem', 'particular', 'council', 'wish', 'take', 'part', 'sufficiently', 'reflexion']
['course', 'one', 'crucial', 'event', 'namely', 'start', 'made', 'category', 'four', 'looking', 'ways', 'deal', 'yet', 'entirely', 'clear', 'whether', 'managed', 'resolve', 'partly', 'council', 'refuses', 'sufficient', 'input', 'attendant', 'thought', 'processes']
['nigeria', 'encouraging', 'violence', 'employment', 'go', 'pay', 'soldiers', 'draw', 'demonstrators', 'basis', 'w

['want', 'help', 'indonesia', 'must', 'intervene', 'money', 'concrete', 'actions', 'words', '.']
['want', 'help', 'indonesia', 'must', 'act', 'money', 'deeds', 'merely', 'words']
['bit', 'compel', 'commission', 'motion', 'resolution', 'ask', 'time', 'establish', 'green', 'paper', 'current', 'state', 'voluntary', 'insurance', 'health', '.']
['find', 'rather', 'odd', 'people', 'already', 'trying', 'tie', 'commission', "'s", 'hands', 'relation', 'proposal', 'directive', 'calling', 'present', 'green', 'paper', 'current', 'situation', 'regard', 'optional', 'supplementary', 'health', 'insurance', 'schemes']
['course', 'important', 'point', 'fact', 'initiated', 'within', 'category', 'four', 'approach', 'yet', 'know', 'well', 'managed', 'solve', 'problem', 'least', 'council', 'wish', 'participate', 'enough', 'food', 'thought']
['course', 'one', 'crucial', 'event', 'namely', 'start', 'made', 'category', 'four', 'looking', 'ways', 'deal', 'yet', 'entirely', 'clear', 'whether', 'managed', 'resolv

['disgrace', 'believed', 'european', 'union', 'set', 'champion', 'modernisation', 'economic', 'life']
['shameful', 'state', 'affairs', 'consider', 'eu', 'champion', 'modernised', 'business', 'practice']
['mr', 'president', 'speech', 'shall', 'focus', 'report', 'mrs', 'lalumière', 'think', 'well', 'thought-out', 'clearly', 'formulated']
['mr', 'president', 'would', 'like', 'focus', 'speech', 'mrs', 'lalumière', "'", 'report', 'think', 'clearly', 'worded', 'well', 'put', 'together']
['company', 'unipersonnelle', 'society', 'relying', 'less', '250', 'employees', 'project', 'must', 'possible', 'receive', 'funding', 'guarantee', '120', '%', 'also', 'simple', 'basis', 'good', 'idea']
['one-man', 'company', 'fewer', '250', 'employees', 'project', 'need', 'provide', '120', '%', 'loan', 'guarantees', 'form', 'land', 'receive', 'financing', ';', 'also', 'possible', 'funded', 'basis', 'good', 'idea']
['must', 'true', 'brain', 'coordination', 'network', 'national', 'agencies', 'turn', 'activate', 

['objectives', 'primary', 'objective', 'secondary', 'represent', 'stone', 'hitting', 'enabling', 'us', 'judge', 'concrete', 'proposals', 'presented', 'commission']
['objectives', 'main', 'one', 'secondary', 'ones', 'touchstone', 'concrete', 'proposals', 'submitted', 'commission', 'must', 'judged']
['contrary', 'power', 'national', 'democracies', 'increasingly', 'restrict', 'right', 'veto', 'giving', 'parliament', 'eu']
['contrary', 'influence', 'national', 'democracies', 'reduced', 'restriction', 'right', 'veto', 'fact', 'european', 'parliament', 'given', 'power']
['party', 'issues', 'serious', 'reservations', 'connection', 'regulation', 'sales', 'products', 'consumption', 'means', 'community', 'legislation', 'like', 'reserves', 'concept', 'mutual', 'recognition', 'standards']
['-', 'party', 'serious', 'reservations', 'community', 'law', 'applying', 'sale', 'consumer', 'products', 'concept', 'mutual', 'recognition', 'standards']
['find', 'france', 'exemplary', 'committed', 'take', 'ste

['thank', 'much', 'mr', 'commissioner', '.']
['thank', 'much', 'commissioner']
['urgency', 'decided', 'put', 'issue', 'agenda']
['action', 'needed', 'quickly', 'decided', 'include', 'item', 'agenda']
['lesson', 'us', 'within', 'house', 'debate', 'morning', 'forced', 'conclude', 'laws', 'sea', 'world', 'state', 'chaos', 'must', 'get', 'work', 'bring', 'order']
['lesson', 'parliament', 'morning', 'must', 'conclude', 'maritime', 'laws', 'throughout', 'world', 'state', 'shambles', 'begin', 'process', 'putting', 'right']
['either', 'part', 'club', 'must', 'therefore', 'abide', 'decision']
['either', 'club', 'particularly', 'important', 'accept']
['right', 'government', 'remove', 'arbitrarily', 'constitution', 'defining', 'characteristic', 'tyranny']
['right', 'government', 'arbitrarily', 'set', 'aside', 'constitution', 'defining', 'characteristic', 'tyranny']
['indeed', 'convinced', 'definition', 'fair', 'price', 'proposed', 'better', 'another', 'different', 'definitions', 'used', 'member',

In [19]:
scores_train[0:5]

['4', '4', '3', '3', '2']

## Models

In this section we fit our feature set to a model.

### Decision Tree

In [20]:
dt_classifier = DecisionTreeClassifier(random_state=14, max_depth=8)
dt_classifier.fit(data,scores_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=8,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=14,
            splitter='best')

In [21]:
print(f'Nodes: {dt_classifier.tree_.node_count}')
print(f'Max Depth: {dt_classifier.tree_.max_depth}')
print(f'Accuracy: {dt_classifier.score(data, scores_train)}')

Nodes: 239
Max Depth: 8
Accuracy: 0.7109164420485176


### XGBoost

In [22]:
xgboost_model = XGBClassifier(booster='gbtree', 
                       n_estimators=1000,
                       n_jobs=4,
                       learning_rate=.05,
                       max_depth=4,
                       random_state=42,
                       gamma=.05,
                       early_stopping_rounds = 5)



In [28]:
xgboost_model = XGBClassifier(n_jobs=4, n_estimators=1000, gamma=.05, random_state=42)
xgboost_model.fit(np.asarray(data), np.asarray(scores_train))

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0.05,
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=1000, n_jobs=4,
       nthread=None, objective='multi:softprob', random_state=42,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=1, verbosity=1)

### Random Forest

In [29]:
rf_classifier = RandomForestClassifier(random_state=14, n_estimators=100)
rf_classifier.fit(data,scores_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=14, verbose=0, warm_start=False)

# Testing

In [ ]:
s1_arr_dev, s2_arr_dev, scores_dev, s1_tokens_dev, s2_tokens_dev, s1_lemmas_dev,s2_lemmas_dev = preprocess("./data/dev-set.txt")
dev_data = pipeline(s1_arr_dev, s2_arr_dev, s1_tokens_dev, s2_tokens_dev, s1_lemmas_dev,s2_lemmas_dev)
dt_dev_predictions = dt_classifier.predict(dev_data)
rf_dev_predictions = rf_classifier.predict(dev_data)
xgb_dev_predictions = xgboost_model.predict(np.asarray(dev_data))
dev_data[0:5]

['leaders', 'benefit', 'aujourd', "'", 'hui', 'new', 'luck', 'let', "'s", 'therefore', 'seize']
['leaders', 'given', 'new', 'chance', 'let', 'us', 'hope', 'seize']
['amendment', '7', 'proposing', 'certain', 'changes', 'references', 'paragraphs']
['amendment', '7', 'proposes', 'certain', 'changes', 'references', 'paragraphs']
['would', 'like', 'remind', 'among', 'allies', 'strong', 'tax']
['let', 'remind', 'allies', 'include', 'fervent', 'supporters', 'tax']
['vote', 'take', 'place', '5.30pm']
['vote', 'take', 'place', 'today', '5.30', 'p.m']
['fishermen', 'inactive', 'tired', 'disappointed']
['fishermen', 'inactive', 'tired', 'disappointed']
['majority', 'voting', 'parliament', 'go', 'back', 'article', '272']
['neither', 'qualified', 'majority', 'within', 'house', 'revert', 'article', '272']
['effect', 'augmenting', 'potency', 'big', 'countries', 'detriment', 'babies']
['increases', 'power', 'big', 'countries', 'expense', 'small']
['fishers', 'inactive', 'tired', 'disappointed']
['fish

['mr', 'van', 'orden', 'report', '(', 'a5-0241', '/', '2000', ')']
['van', 'orden', 'report', '(', 'a5-0241/2000', ')']
['mr.', 'president', 'cashman', 'report/ratio', 'summarize', 'words', ':', 'capacity', 'citizens', 'bureaucracy']
['mr', 'president', 'cashman', 'report', 'summarised', 'four', 'words', ':', 'citizens', "'", 'power', 'bureaucracy']
['amendment', '7', 'proposes', 'certain', 'modifications', 'references', 'paragraphs']
['amendment', '7', 'proposes', 'certain', 'changes', 'references', 'paragraphs']
['users', 'losers', 'employees', 'european', 'competitiveness', 'growth', 'régresseront']
['consumers', 'lose', 'employees', 'europe', 'competitive', 'strength', 'growth']
['duty', 'continue', 'support', 'latvia', 'question', 'integration', 'russian', 'population']
['job', 'continue', 'support', 'latvia', 'integration', 'russian', 'population']
['(', 'parliament', 'adopted', 'legislative', 'resolution', ')']
['(', 'parliament', 'adopted', 'legislative', 'resolution', ')']
['r

['maybe', 'could', 'avoided', 'catastrophe']
['perhaps', 'could', 'avoided', 'catastrophe']
['tunisia']
['tunisia']
['mr', 'president', 'report', 'cashman', "'s", 'summarised', 'words', ':', 'power', 'people', 'tape']
['mr', 'president', 'cashman', 'report', 'summarised', 'four', 'words', ':', 'citizens', "'", 'power', 'bureaucracy']
['unanimity', 'council', 'therefore', 'covert', 'inability', 'act']
['unanimous', 'decisions', 'hence', 'inherent', 'incapacity', 'act', 'remain', 'largely', 'norm', 'council']
['fishermen', 'inactive', 'tired', 'disappointed']
['fishermen', 'inactive', 'tired', 'disappointed']
['mister', 'president', 'anxious', 'take', 'defense', 'workers', 'sector']
['mr', 'president', 'rise', 'defence', 'workers', 'industry']
['(', 'parliament', 'adopts', 'legislative', 'resolution', ')']
['(', 'parliament', 'adopted', 'legislative', 'resolution', ')']
['users', 'losers', 'employees', 'european', 'competitiveness', 'growth', 'régresseront']
['consumers', 'lose', 'employ

['part', 'parliamentarians', 'rather', 'present', 'honourably']
['one', 'members', 'attends', 'sittings', 'quite', 'faithfully']
['mister', 'president', 'report', 'cashman', 'summarize', 'words', ':', 'strength', 'citizens', 'bureaucracy']
['mr', 'president', 'cashman', 'report', 'summarised', 'four', 'words', ':', 'citizens', "'", 'power', 'bureaucracy']
['european', 'union', 'involved', 'soon']
['european', 'union', 'got', 'something', 'quickly']
['one', 'members', 'quite', 'honourably']
['one', 'members', 'attends', 'sittings', 'quite', 'faithfully']
['fishermen', 'inactive', 'tired', 'disappointed']
['fishermen', 'inactive', 'tired', 'disappointed']
['right', 'european', 'union', 'entails', 'quickly']
['european', 'union', 'got', 'something', 'quickly']
['mr', 'president', 'want', 'stand', 'workers', 'area']
['mr', 'president', 'rise', 'defence', 'workers', 'industry']
['unit', 'must', 'balance']
['must', 'balance', 'whole']
['keep', 'l', "'", 'unanimity', 'council', 'therefore', '

['others', 'unfortunately', 'diverge', 'response', 'rancune', 'gained']
['unfortunately', 'others', 'separate', 'basis', 'accumulated', 'hatred']
['fishermen', 'inactive', 'tired', 'disappointed']
['fishermen', 'inactive', 'tired', 'disappointed']
['mr', 'president', 'wish', 'take', 'protection', 'workers', 'sector']
['mr', 'president', 'rise', 'defence', 'workers', 'industry']
['vote', 'take', 'place', '17', ':', '30']
['vote', 'take', 'place', 'today', '5.30', 'p.m']
['standards', 'hardly', 'comparable', 'still', 'less', 'transferable']
['standards', 'scarcely', 'comparable', 'let', 'alone', 'transferable']
['effect', 'increasing', 'power', 'large', 'countries', 'detriment', 'small']
['increases', 'power', 'big', 'countries', 'expense', 'small']
['could', 'perhaps', 'able', 'prevent', 'disaster']
['perhaps', 'could', 'avoided', 'catastrophe']
['whole', 'must', 'balance']
['must', 'balance', 'whole']
['already', 'explained', 'us', 'second', 'reading', 'crisis', 'root', 'amendment', 'd

['point', 'procedure', 'opens', 'door', 'arbitrary']
['provision', 'could', 'open', 'door', 'wide', 'arbitrariness']
['qualified', 'majority', 'parliament', 'go', 'back', 'article', '272']
['neither', 'qualified', 'majority', 'within', 'house', 'revert', 'article', '272']
['specific', 'aid', 'market', 'premium', 'grass', 'essential']
['selective', 'aid', 'market', 'support', 'grass', 'subsidy', 'essential']
['duty', 'continue', 'support', 'latvia', 'issue', 'integration', 'russian', 'people']
['job', 'continue', 'support', 'latvia', 'integration', 'russian', 'population']
['vote', 'take', 'place', '17h30']
['vote', 'take', 'place', 'today', '5.30', 'p.m']
['could', 'perhaps', 'avoided', 'catastrophe']
['perhaps', 'could', 'avoided', 'catastrophe']
['tunisia']
['tunisia']
['standards', 'hardly', 'comparable', 'still', 'less', 'transferable']
['standards', 'scarcely', 'comparable', 'let', 'alone', 'transferable']
['might', 'avoid', 'disaster']
['perhaps', 'could', 'avoided', 'catastrophe

['already', 'explained', 'second', 'reading', 'crisis', 'located', 'basis', 'modification', 'directive']
['already', 'explained', 'second', 'reading', 'crisis', 'underlying', 'directive', 'amendment']
['report/ratio', 'maij-weggen', '(', 'a5-0323/2000', ')']
['maij-weggen', 'report', '(', 'a5-0323/2000', ')']
['make', 'point', 'recalling', 'among', 'allies', 'enthusiasts', 'tax']
['let', 'remind', 'allies', 'include', 'fervent', 'supporters', 'tax']
['users', 'losers', 'employees', 'european', 'competitiveness', 'growth', 'diminish']
['consumers', 'lose', 'employees', 'europe', 'competitive', 'strength', 'growth']
['already', 'explained', 'second', 'reading', 'crisis', 'basis', 'amendment', 'directive']
['already', 'explained', 'second', 'reading', 'crisis', 'underlying', 'directive', 'amendment']
['users', 'losers', 'employees', 'european', 'competitiveness', 'growth', 'diminish']
['consumers', 'lose', 'employees', 'europe', 'competitive', 'strength', 'growth']
['mr', 'president', 'wo

['butterflies', 'housed', 'earlier', 'light', 'cycle', '1', 'a.m.', 'p.m.', 'flew', 'toward', 'southeast']
['butterflies', 'exposed', 'earlier', 'light', 'cycle', '1am', '1pm', 'orientated', 'towards', 'south-east']
['good', 'reputation', "''", 'rt', 'jones', 'analyst', 'juli', 'niemann', 'said', 'grant']
['rt', 'jones', 'analyst', 'juli', 'niemann', 'said', 'grant', 'one', 'pulling']
['france', 'infuriated', 'washington', 'leading', 'charge', 'u.n.', 'approval', 'war', 'also', 'sought', 'changes']
['key', 'question', 'whether', 'france', 'infuriated', 'washington', 'leading', 'charge', 'u.n.', 'authorization', 'war', 'would', 'vote', 'yes', "''", 'abstain']
['mel', 'gibson', 'negotiating', 'newmarket', 'films', 'distribute', 'embattled', 'biblical', 'epic', 'passion', 'christ', "''", 'united', 'states']
['mel', 'gibson', "'s", 'passion-stirring', 'biblical', 'epic', 'passion', 'christ', "''", 'open', 'united', 'states', 'feb.', '25', '-', 'ash', 'wednesday', 'roman', 'catholic', 'cale

['wire', 'transfers', 'four', '40', 'accounts', 'open', 'beacon', 'hill', 'totaled', '$', '3.2', 'billion', '2001', '2002', 'morgenthau', 'said']
['2001', '2002', 'wire', 'transfers', '4', 'company', "'s", '40', 'accounts', 'totaled', '$', '3.2', 'billion', 'prosecutors', 'said']
['state', 'university', 'stony', 'brook', 'launched', 'study', '1996', 'earlier', 'studies', 'indicated', 'possible', 'connection']
['stony', 'brook', 'university', 'launched', 'study', '1996', 'earlier', 'studies', 'indicated', 'possible', 'connection', 'electromagnetic', 'fields', 'cancer']
['earlier', 'bashir', "'s", 'supporters', 'defied', 'police', 'order', 'cried', 'allahu', 'akbar', '(', 'god', 'greatest', ')', "''", 'walked', 'seat']
['earlier', 'defied', 'police', 'order', 'cried', 'allahu', 'akbar', "''", '(', 'god', 'greatest', ')', 'bashir', 'walked', 'seat', 'tightly', 'guarded', 'courtroom']
['spokesman', 'james', 'howe', 'said', 'five', 'children', 'aged', '4', '17', 'taken', 'hospital', 'neck',

In [ ]:
# make sure our lengths match up
print(len(scores_dev))
print(len(dt_dev_predictions))
print(len(rf_dev_predictions))
print(len(xgb_dev_predictions))

In [ ]:
def calc_accuracy(predictions, scores):
    correct = 0
    for i in range(0, len(predictions)): 
        if predictions[i] == scores[i]:
            correct += 1
    return correct / len(predictions) 



    
print('DECISION TREE: ', calc_accuracy(dt_dev_predictions, scores_dev))
print('RANDOM FOREST: ', calc_accuracy(rf_dev_predictions, scores_dev))
print('XGBOOST: ', calc_accuracy(xgb_dev_predictions, scores_dev))